In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device( 'cpu')


In [3]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'


In [4]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [5]:
# Image processing
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
#                                      std=(0.5, 0.5, 0.5))])

In [6]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

In [7]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

In [8]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [9]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())


In [10]:
# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [11]:

# Device setting
D = D.to(device)
G = G.to(device)

In [12]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [13]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [14]:
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [15]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))


Epoch [0/200], Step [200/600], d_loss: 0.0844, g_loss: 3.4066, D(x): 0.99, D(G(z)): 0.07
Epoch [0/200], Step [400/600], d_loss: 0.0293, g_loss: 5.6871, D(x): 0.99, D(G(z)): 0.02
Epoch [0/200], Step [600/600], d_loss: 0.0649, g_loss: 6.0386, D(x): 0.99, D(G(z)): 0.05
Epoch [1/200], Step [200/600], d_loss: 0.0838, g_loss: 4.0978, D(x): 0.96, D(G(z)): 0.04
Epoch [1/200], Step [400/600], d_loss: 0.4715, g_loss: 5.2037, D(x): 0.80, D(G(z)): 0.07
Epoch [1/200], Step [600/600], d_loss: 0.0982, g_loss: 4.7979, D(x): 0.96, D(G(z)): 0.04
Epoch [2/200], Step [200/600], d_loss: 0.1623, g_loss: 4.6401, D(x): 0.93, D(G(z)): 0.06
Epoch [2/200], Step [400/600], d_loss: 0.2850, g_loss: 4.0721, D(x): 0.92, D(G(z)): 0.15
Epoch [2/200], Step [600/600], d_loss: 0.2365, g_loss: 3.3498, D(x): 0.93, D(G(z)): 0.10
Epoch [3/200], Step [200/600], d_loss: 0.2883, g_loss: 4.7200, D(x): 0.93, D(G(z)): 0.13
Epoch [3/200], Step [400/600], d_loss: 0.4086, g_loss: 4.0852, D(x): 0.91, D(G(z)): 0.20
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.2927, g_loss: 4.0652, D(x): 0.95, D(G(z)): 0.14
Epoch [31/200], Step [200/600], d_loss: 0.3672, g_loss: 4.3435, D(x): 0.88, D(G(z)): 0.06
Epoch [31/200], Step [400/600], d_loss: 0.2603, g_loss: 5.5234, D(x): 0.88, D(G(z)): 0.04
Epoch [31/200], Step [600/600], d_loss: 0.1992, g_loss: 5.5267, D(x): 0.90, D(G(z)): 0.04
Epoch [32/200], Step [200/600], d_loss: 0.4193, g_loss: 4.3144, D(x): 0.86, D(G(z)): 0.05
Epoch [32/200], Step [400/600], d_loss: 0.3772, g_loss: 3.5615, D(x): 0.85, D(G(z)): 0.04
Epoch [32/200], Step [600/600], d_loss: 0.3202, g_loss: 5.1115, D(x): 0.88, D(G(z)): 0.06
Epoch [33/200], Step [200/600], d_loss: 0.2950, g_loss: 4.8138, D(x): 0.91, D(G(z)): 0.07
Epoch [33/200], Step [400/600], d_loss: 0.2135, g_loss: 3.5627, D(x): 0.97, D(G(z)): 0.13
Epoch [33/200], Step [600/600], d_loss: 0.4740, g_loss: 4.8084, D(x): 0.86, D(G(z)): 0.04
Epoch [34/200], Step [200/600], d_loss: 0.2602, g_loss: 3.7656, D(x): 0.91, D(G(z)): 0.05
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.5866, g_loss: 2.2443, D(x): 0.80, D(G(z)): 0.20
Epoch [61/200], Step [600/600], d_loss: 0.6014, g_loss: 2.5340, D(x): 0.79, D(G(z)): 0.18
Epoch [62/200], Step [200/600], d_loss: 0.8402, g_loss: 1.5313, D(x): 0.79, D(G(z)): 0.30
Epoch [62/200], Step [400/600], d_loss: 0.6920, g_loss: 3.0238, D(x): 0.82, D(G(z)): 0.25
Epoch [62/200], Step [600/600], d_loss: 0.5530, g_loss: 2.5994, D(x): 0.82, D(G(z)): 0.19
Epoch [63/200], Step [200/600], d_loss: 0.7604, g_loss: 2.6845, D(x): 0.79, D(G(z)): 0.23
Epoch [63/200], Step [400/600], d_loss: 0.5702, g_loss: 2.7461, D(x): 0.80, D(G(z)): 0.20
Epoch [63/200], Step [600/600], d_loss: 0.9032, g_loss: 2.4540, D(x): 0.71, D(G(z)): 0.21
Epoch [64/200], Step [200/600], d_loss: 0.5511, g_loss: 2.9361, D(x): 0.84, D(G(z)): 0.21
Epoch [64/200], Step [400/600], d_loss: 0.7251, g_loss: 2.1723, D(x): 0.82, D(G(z)): 0.30
Epoch [64/200], Step [600/600], d_loss: 0.5730, g_loss: 2.3471, D(x): 0.80, D(G(z)): 0.17
Epoch [65/

Epoch [92/200], Step [200/600], d_loss: 0.6443, g_loss: 2.4125, D(x): 0.78, D(G(z)): 0.23
Epoch [92/200], Step [400/600], d_loss: 0.9168, g_loss: 1.8931, D(x): 0.74, D(G(z)): 0.31
Epoch [92/200], Step [600/600], d_loss: 0.9250, g_loss: 1.9891, D(x): 0.71, D(G(z)): 0.29
Epoch [93/200], Step [200/600], d_loss: 0.7644, g_loss: 2.0089, D(x): 0.72, D(G(z)): 0.24
Epoch [93/200], Step [400/600], d_loss: 0.7568, g_loss: 2.1036, D(x): 0.79, D(G(z)): 0.30
Epoch [93/200], Step [600/600], d_loss: 0.8536, g_loss: 1.9145, D(x): 0.72, D(G(z)): 0.26
Epoch [94/200], Step [200/600], d_loss: 0.8250, g_loss: 1.4669, D(x): 0.71, D(G(z)): 0.26
Epoch [94/200], Step [400/600], d_loss: 1.1420, g_loss: 2.4042, D(x): 0.57, D(G(z)): 0.18
Epoch [94/200], Step [600/600], d_loss: 0.7803, g_loss: 1.6588, D(x): 0.70, D(G(z)): 0.23
Epoch [95/200], Step [200/600], d_loss: 0.7776, g_loss: 1.6978, D(x): 0.70, D(G(z)): 0.22
Epoch [95/200], Step [400/600], d_loss: 0.8865, g_loss: 2.3144, D(x): 0.69, D(G(z)): 0.19
Epoch [95/

Epoch [122/200], Step [400/600], d_loss: 0.8376, g_loss: 1.8113, D(x): 0.75, D(G(z)): 0.30
Epoch [122/200], Step [600/600], d_loss: 0.9278, g_loss: 1.7570, D(x): 0.77, D(G(z)): 0.35
Epoch [123/200], Step [200/600], d_loss: 1.1385, g_loss: 1.9740, D(x): 0.63, D(G(z)): 0.31
Epoch [123/200], Step [400/600], d_loss: 0.8403, g_loss: 1.9657, D(x): 0.76, D(G(z)): 0.31
Epoch [123/200], Step [600/600], d_loss: 0.6400, g_loss: 2.4487, D(x): 0.72, D(G(z)): 0.15
Epoch [124/200], Step [200/600], d_loss: 1.0490, g_loss: 1.9031, D(x): 0.59, D(G(z)): 0.16
Epoch [124/200], Step [400/600], d_loss: 0.9635, g_loss: 1.6445, D(x): 0.75, D(G(z)): 0.31
Epoch [124/200], Step [600/600], d_loss: 0.7463, g_loss: 1.8676, D(x): 0.69, D(G(z)): 0.18
Epoch [125/200], Step [200/600], d_loss: 0.9483, g_loss: 1.5967, D(x): 0.65, D(G(z)): 0.28
Epoch [125/200], Step [400/600], d_loss: 0.9948, g_loss: 2.1730, D(x): 0.62, D(G(z)): 0.21
Epoch [125/200], Step [600/600], d_loss: 0.7431, g_loss: 1.8597, D(x): 0.80, D(G(z)): 0.29

Epoch [152/200], Step [600/600], d_loss: 0.9661, g_loss: 1.7969, D(x): 0.65, D(G(z)): 0.27
Epoch [153/200], Step [200/600], d_loss: 0.8762, g_loss: 1.2360, D(x): 0.68, D(G(z)): 0.29
Epoch [153/200], Step [400/600], d_loss: 1.0032, g_loss: 1.8313, D(x): 0.63, D(G(z)): 0.30
Epoch [153/200], Step [600/600], d_loss: 1.0382, g_loss: 1.6903, D(x): 0.72, D(G(z)): 0.39
Epoch [154/200], Step [200/600], d_loss: 1.0181, g_loss: 1.3976, D(x): 0.74, D(G(z)): 0.39
Epoch [154/200], Step [400/600], d_loss: 0.9998, g_loss: 1.4280, D(x): 0.67, D(G(z)): 0.28
Epoch [154/200], Step [600/600], d_loss: 1.0616, g_loss: 1.6136, D(x): 0.63, D(G(z)): 0.31
Epoch [155/200], Step [200/600], d_loss: 1.0622, g_loss: 1.4738, D(x): 0.68, D(G(z)): 0.33
Epoch [155/200], Step [400/600], d_loss: 0.9103, g_loss: 1.7853, D(x): 0.76, D(G(z)): 0.36
Epoch [155/200], Step [600/600], d_loss: 1.0396, g_loss: 1.6571, D(x): 0.67, D(G(z)): 0.34
Epoch [156/200], Step [200/600], d_loss: 1.0110, g_loss: 1.5963, D(x): 0.66, D(G(z)): 0.31

Epoch [183/200], Step [200/600], d_loss: 1.0747, g_loss: 1.9283, D(x): 0.60, D(G(z)): 0.26
Epoch [183/200], Step [400/600], d_loss: 0.8668, g_loss: 1.7584, D(x): 0.69, D(G(z)): 0.29
Epoch [183/200], Step [600/600], d_loss: 1.0150, g_loss: 1.3595, D(x): 0.63, D(G(z)): 0.30
Epoch [184/200], Step [200/600], d_loss: 0.7328, g_loss: 1.5475, D(x): 0.76, D(G(z)): 0.28
Epoch [184/200], Step [400/600], d_loss: 1.0342, g_loss: 1.4931, D(x): 0.63, D(G(z)): 0.30
Epoch [184/200], Step [600/600], d_loss: 1.0981, g_loss: 1.3449, D(x): 0.68, D(G(z)): 0.40
Epoch [185/200], Step [200/600], d_loss: 1.0104, g_loss: 1.5387, D(x): 0.66, D(G(z)): 0.31
Epoch [185/200], Step [400/600], d_loss: 0.9471, g_loss: 1.3866, D(x): 0.71, D(G(z)): 0.33
Epoch [185/200], Step [600/600], d_loss: 0.9512, g_loss: 1.5028, D(x): 0.71, D(G(z)): 0.35
Epoch [186/200], Step [200/600], d_loss: 0.9446, g_loss: 1.8071, D(x): 0.71, D(G(z)): 0.35
Epoch [186/200], Step [400/600], d_loss: 1.0489, g_loss: 1.4067, D(x): 0.68, D(G(z)): 0.37

In [16]:
# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')